<a href="https://colab.research.google.com/github/sid0nair/3D-CNN-/blob/main/STL_to_voxel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stl-to-voxel

In [3]:
!pip install trimesh

import trimesh
import numpy as np
import os
from glob import glob

input_folder  = "/content/drive/MyDrive/STL files motor"
output_folder = "/content/drive/MyDrive/voxel_grids"
os.makedirs(output_folder, exist_ok=True)

GRID_DIM = 64


def pad_to_shape(mat, tgt):
    cur = mat.shape
    pads = []
    for c, t in zip(cur, (tgt,)*3):
        if c < t:
            total = t - c
            before = total // 2
            after  = total - before
        else:
            before = 0
            after  = 0
        pads.append((before, after))
    mat = np.pad(mat, pads, mode='constant', constant_values=False)
    # if any dim > tgt, crop evenly
    return mat[
        :tgt,
        :tgt,
        :tgt
    ]

# 6) Process every .stl in the folder
stl_paths = glob(os.path.join(input_folder, "*.stl"))
for stl_path in stl_paths:
    # load mesh
    mesh = trimesh.load_mesh(stl_path, force='mesh')
    # compute a pitch so the largest bbox axis becomes GRID_DIM voxels
    mins, maxs = mesh.bounds
    extents = maxs - mins
    pitch = extents.max() / GRID_DIM

    # voxelize
    vox = mesh.voxelized(pitch)
    mat = vox.matrix.astype(bool)  # 3D boolean array

    # pad / crop to uniform size
    mat = pad_to_shape(mat, GRID_DIM)

    # save
    name = os.path.splitext(os.path.basename(stl_path))[0]
    out_path = os.path.join(output_folder, f"{name}.npy")
    np.save(out_path, mat)
    print(f"saved {out_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 711.2/711.2 kB 9.4 MB/s eta 0:00:00
→ saved /content/drive/MyDrive/voxel_grids/2tk3a-aw2u.npy
→ saved /content/drive/MyDrive/voxel_grids/4tk10a-aw2u.npy
→ saved /content/drive/MyDrive/voxel_grids/3tk6a-aw2u.npy
→ saved /content/drive/MyDrive/voxel_grids/5tk20a-cw2e.npy
→ saved /content/drive/MyDrive/voxel_grids/5tk20a-aw2u.npy
→ saved /content/drive/MyDrive/voxel_grids/4tk10a-cw2e.npy
→ saved /content/drive/MyDrive/voxel_grids/2tk3a-cw2e.npy
→ saved /content/drive/MyDrive/voxel_grids/3tk6a-cw2e.npy
→ saved /content/drive/MyDrive/voxel_grids/3rk15a-ua.npy
→ saved /content/drive/MyDrive/voxel_grids/3rk15a-uam.npy
→ saved /content/drive/MyDrive/voxel_grids/3rk15a-uc.npy
→ saved /content/drive/MyDrive/voxel_grids/3rk15a-ucm.npy
All done!
